# (1) description of dataset
### players.csv
contains survey data of players self declared experience, whether they are subscribed, their hashed emails a form of ID, self-reported play hours, name, gender and age.

Notes: 
1. there exists people who participated in the survey who did not log in to the server even once.
2. people may be lying in the survey (there is a bit of time discrepency between self reported play time and actual accumulated session play time)
3. individualID and organization name columns do not seem to have any useful or relavent data
4. each hashedEmail only appears once, so it can be used to key the data.

### sessions.csv
contains session data (presumably from minecraft logs) of each players play session, including the player hashedEmail as a form of ID, start time, end time (to nearest minute) and two columns of garbage data supposed to be some sort of epoch time but messed up, we will not be using this.

- hashedEmail: string, type of ID for each player (not a suitable key for the table)
- start_time: string for start time/log on time (to nearest minute)
- end_time: string for end time/log off time (to nearest minute)
- original_start_time/original_end_time: broken epoch time of start/end of session


Notes:
1. contains rows where start and end time is not recorded
2. a single player may log on multiple sessions, so hashedEmail is not a UID for each row.


# (2) Question

We will be answering question 3. specifically, can we predict the length of a players session given their join time (which day of the week, and at what hour they are joining). 

Predictor variable(s): Minutes since the start of the week
Response/output variable(s): predicted amount of time the player stays logged on the server

Learnt Method to use: KNN regression. We are not classifying anything, and since the data is perliminarily visualized to be wave-like in nature, the predictor model should be able to curve (so we will not use linear regression)



# (3) Exploratory visualization
(see below)

# (4) My plan:
### what data to use?
I will be using sessions.csv, because players.csv does not give any useful information, is noisy and is inconsistent with the session data.

The hashedEmail column is also not useful, we are not predicting activity on a per player basis (although we could, there is not enough data)

### the actual plan to do stuff...
1. use start and end time columns to produce relavent epoch time columns.
2. apply modulus epoch time columns to get "seconds since start of week" column
3. plot player count (y axis) vs time of week
4. train KNN regressor on the data.


### Why is this method appropriate?
  we are using a scalar predictor to produce a scalar output that is not known to be linear by nature. It is not a classification problem, so we will use a regressor. KNN regressor is not linear, so it will be our choice.
### Which assumptions are required, if any, to apply the method selected?
  n/a
### What are the potential limitations or weaknesses of the method selected?
  might encounter over/underfitting
  Model might not scale well (but is appropriate for such a small dataset we are using)
### How are you going to compare and select the model?
  25% of the data will be randomly selected to test the model of N neighbours, we can run it as many times as we want, randomizing the split each time, to properly gague average model performance.
  We might also compare it to a linear regression model if the data resembles anything linear.
### How are you going to process the data to apply the model? For example: Are you splitting the data? How? How many splits? What proportions will you use for the splits? At what stage will you split? Will there be a validation set? Will you use cross validation?
  1. by turning sessions data into a simple linear dataset of players/time
  2. yes, randomly multiple times, 25%.
  3. split when the data is completely wrangled
  4. yes
  5. yes, as many times as computationally fesible




In [1]:
# All necessary library imports?
import pandas as pd
import numpy as np
import altair as alt
from datetime import datetime

from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Simplify working with large datasets in Altair
alt.data_transformers.enable('vegafusion')

# Output dataframes instead of arrays
set_config(transform_output="pandas")

In [2]:
sessions_df = pd.read_csv("https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB")
# Import the sessions CSV from drive link

In [3]:
sessions_df.head(5)
# Show a bit of the sessions dataframe

,hashedEmail,start_time,end_time,original_start_time,original_end_time
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,30/06/2024 18:12,30/06/2024 18:24,1.719770e+12,1.719770e+12
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,17/06/2024 23:33,17/06/2024 23:46,1.718670e+12,1.718670e+12
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,25/07/2024 17:34,25/07/2024 17:57,1.721930e+12,1.721930e+12
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,25/07/2024 03:22,25/07/2024 03:58,1.721880e+12,1.721880e+12
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,25/05/2024 16:01,25/05/2024 16:12,1.716650e+12,1.716650e+12


In [4]:
format_code = "%d/%m/%Y %H:%M"

global i;
i = 0

def catchWrongDate(d):
    global i;
    s = str(d)
    if(len(s) != 16):
        pos = 11
        s = s[:pos] + "0" + s[pos:]
    return(s)


sessions_df.loc[sessions_df['end_time'] == '', 'end_time'] = np.nan # if end_time is empty, turn it to NaNs
sessions_df.dropna(subset=['end_time'], inplace=True) # drop all NaNs
sessions_df.reset_index(inplace=True)

sessions_df['time'] = sessions_df.apply(
    lambda row: (datetime.strptime(catchWrongDate(row["end_time"]), format_code)-datetime.strptime(catchWrongDate(row["start_time"]), format_code)).total_seconds(), 
    axis=1
)

sessions_df["start_time"] = pd.to_datetime(sessions_df['start_time'],format="%d/%m/%Y %H:%M")

sessions_df['start_hr_of_day'] = sessions_df["start_time"].dt.hour
sessions_df['start_day_of_week'] = sessions_df["start_time"].dt.weekday
sessions_df['start_hr_of_week'] = sessions_df['start_hr_of_day']+(24*sessions_df['start_day_of_week'])

sessions_df.head(5)


,index,hashedEmail,start_time,end_time,original_start_time,original_end_time,time,start_hr_of_day,start_day_of_week,start_hr_of_week
0,0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-06-30 18:12:00,30/06/2024 18:24,1.719770e+12,1.719770e+12,720.0,18,6,162
1,1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-06-17 23:33:00,17/06/2024 23:46,1.718670e+12,1.718670e+12,780.0,23,0,23
2,2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,2024-07-25 17:34:00,25/07/2024 17:57,1.721930e+12,1.721930e+12,1380.0,17,3,89
3,3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-07-25 03:22:00,25/07/2024 03:58,1.721880e+12,1.721880e+12,2160.0,3,3,75
4,4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-05-25 16:01:00,25/05/2024 16:12,1.716650e+12,1.716650e+12,660.0,16,5,136


!!! ^^ explain what we're doing with 'def' --  defining function, what it does

In [5]:
sessions_grouped = sessions_df.groupby("hashedEmail").size().reset_index(name='counts')
sessions_grouped #sessions grouped (just for looking)

,hashedEmail,counts
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,2
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,1
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,1
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,13
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,2
...,...,...
120,fc0224c81384770e93ca717f32713960144bf0b52ff676...,1
121,fcab03c6d3079521e7f9665caed0f31fe3dae6b5ccb86e...,1
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,310
123,fe218a05c6c3fc6326f4f151e8cb75a2a9fa29e22b110d...,1


In the original dataframe, there are repeats in `hashedEmail`, which makese sense, as observations represent a log-ons, and thus players, represented through unique `hashedEmail`, can have multiple counts (sessions).

In [6]:
hours = alt.Chart(sessions_df).mark_bar().encode(
    x=alt.X("start_hr_of_day:O",title="Hours after 12 a.m."),
    y=alt.Y("count()",title="Unscaled relative user log-ons")
).properties(
    title="Unscaled cumulative frequency of logins at given time of day"
)
hours

alt.Chart(...)

Based on the above bar graph, bearing in mind that this data set uses 24-hour time, the server's activity peaks in the middle of the night, and lowers during daylight hours. Then in the afternoon to the later evening, we see an increase in activity. Overall, this trend reflects a strong time-of-day effect on player behavior, with distinct periods of high and low number of logins in a day. This gives us a fun glimpse into the average UBC minecrafter's sleep priorities.

# change !!! DELETE THIS, BUT PUT COMMEINT IN CODE CELL BELOW BRIEFLY EXPLAINING WHAT VISUALIZATION IS, like they do in worksheets and shit (graph below, not above)
!!! capitalize title -- capitalize Monday

In [7]:
days = alt.Chart(sessions_df).mark_bar().encode(
    x=alt.X("start_day_of_week:O",title="Days since monday"),
    y=alt.Y("count()",title="unscaled relative user log-ons")
).properties(
    title="unscaled cumulative frequency of logins at given day of week"
)
days

alt.Chart(...)

This bar graph illustrates the number of logins per day, 0 being Monday, and 6 being Sunday. From this we can gather that the server is most active on the weekends, which makes sense, as players have more time to play, not having school or work. Interestingly, however, players are least active on Friday.

From our earlier graph, under the assumption that players are most likely to log on late at night (in the early A.M.), this visualization makes more sense, as most players log on, overall, from 11pm - 4am, which explains the skew between Friday and Saturday -- the days with the respective least and most activity.

Since the data starts from April and ends in September (172.925 days), the day of the week may not have as high as an impact as if it were to be a study done only during the winter sessions.

!!! standardize how we write am (am, a.m. , AM, A.M.?)
!!! should we rename 0-6 Mon through Sun?
0 is monday and so on ... 

In [8]:
main_df = pd.DataFrame({
    "session_play_time":sessions_df["time"]/60/60,
    "start_hour_of_day":sessions_df["original_start_time"]%86400000 / 3600000,
    "end_hour_of_day":sessions_df["original_end_time"]%86400000 / 3600000,
    "start_hour_of_week":sessions_df["original_start_time"]%604800000 / 3600000,
    "end_hour_of_week":sessions_df["original_end_time"]%604800000 / 3600000,

})

main_df

,session_play_time,start_hour_of_day,end_hour_of_day,start_hour_of_week,end_hour_of_week
0,0.200000,17.888889,17.888889,89.888889,89.888889
1,0.216667,0.333333,0.333333,120.333333,120.333333
2,0.383333,17.888889,17.888889,17.888889,17.888889
3,0.600000,4.000000,4.000000,4.000000,4.000000
4,0.183333,15.222222,15.222222,63.222222,63.222222
...,...,...,...,...,...
1528,0.100000,22.444444,22.444444,46.444444,46.444444
1529,0.183333,5.000000,5.000000,101.000000,101.000000
1530,0.350000,15.333333,15.333333,87.333333,87.333333
1531,0.116667,6.777778,6.777778,6.777778,6.777778


!!! ^^ explain all of the division of numbers to put variables into right formats (e.g. divide by this amount to get this time frame) - how we standardize the yucky given data.

In [9]:
def regress(x,x_axis_label="hours since",graph_title="untitled",y="session_play_time"):
    np.random.seed(2025) # to maintain replicability even if you rerun one cell only
    
    param_grid = {
        "kneighborsregressor__n_neighbors": range(1, 201, 1),
    }
    # Search for optimal n neighbors model from 1 to 200 neighbors
    
    tuned = GridSearchCV(
        estimator=pipe, 
        param_grid=param_grid, 
        cv=5,
        n_jobs=-1,
        scoring="neg_root_mean_squared_error",
    )
    # 5 fold cross validation with negative RMSE for scoring
        

    results = pd.DataFrame(tuned.fit(X_train, y_train).cv_results_) 
    
    local_min = tuned.best_params_
    best_RMSPE = -tuned.best_score_
    print(local_min)
    
    
    
    predicted_df = training.assign(
        predictions = tuned.predict(X_train)
    )
    # created predictions for 
    
    marathon_plot = alt.Chart(predicted_df,title=graph_title).mark_circle(opacity=0.4).encode(
        x = alt.X(x,title=x_axis_label),
        y = alt.Y(y,title="session length (hours)")
    )+ alt.Chart(predicted_df).mark_line(color='black').encode(
        x=alt.X(x),
        y=alt.Y("predictions")
    )
    
    
    return(marathon_plot)

^^^ replace marathon stuff with general plot, go into a bit more detail how we're defining function ... 

In [10]:
alt.Chart(main_df).mark_point().encode(
    x=alt.X("start_hour_of_day",title="Start hour of day (hours since 12 a.m.)"),
    y=alt.Y("session_play_time",title="Time played (hours)"),
).properties(
    title="Playtime vs Time Started",width=600
)

alt.Chart(...)

This scatterplot displays each observation's (individual login's) time played versus the hour of the day. It mirrors the first bar graph, in that the times with the highest density of observations here make up each bar in our very first bar graph.

From this scatterplot, the longest sessions are played mostly in the evening, deep into the night.

In [11]:
# alt.Chart(main_df).mark_point().encode(
#     x=alt.X("startw",title="start hr of day"),
#     y=alt.Y("time",title="time played (s)"),
#     ).properties(title="playtime vs time started")

In [12]:
training, testing = train_test_split( 
    main_df,
    test_size=0.25, 
    random_state=2000, # Set random state for replicability
) # Split training and testing data (75%)

X_train = training[["start_hour_of_day"]]  # A single column data frame
y_train = training["session_play_time"]  # A series

X_test = testing[["start_hour_of_day"]]  # A single column data frame
y_test = testing["session_play_time"]  # A series

In [19]:
pipe = make_pipeline(
    StandardScaler(), # Standardize and scale predictors
    KNeighborsRegressor(),
)

session_cv = pd.DataFrame(
    cross_validate(
        estimator=pipe,
        X=X_train,
        y=y_train,
        cv=5,
        scoring="neg_root_mean_squared_error",
        return_train_score=True,
    )
)

session_cv

,fit_time,score_time,test_score,train_score
0,0.004097,0.002413,-1.043066,-0.823191
1,0.003300,0.002281,-0.931811,-0.854639
2,0.003091,0.002256,-0.986957,-0.870458
3,0.002996,0.002187,-0.872028,-0.878992
4,0.002996,0.002170,-1.014720,-0.824356


Because the RMSE comes out to approximately 0.8-1 across our folds, from `test_score`, our model's predictions are off by about 0.8-1 hour. From this, we can say that our model does not perform particularly well.


!!! comment briefly then talk more about at the end (discussion -- it's meh, not that good) 

!!! make last like session_cv["test_score"] = -session_cv["test_score"]
then add brief comment right above it "#multiply test_score by -1 to give absolute value"



In [14]:
regress("start_hour_of_day","Starting hour (hours since 12 a.m.)","Session length vs Start time of day")

{'kneighborsregressor__n_neighbors': 79}


alt.LayerChart(...)

This visualization shows, using our optimal K = 79, the regression line of our model. Looking at our observations, the points,
- data noisy
- overfitting -- jagged and shit

training and testing below is for weeks -- it is useless and disregard?? !!!

In [15]:
# alt.Chart(main_df).mark_point().encode(
#     x=alt.X("endw",title="start hour of day (hours since 12 a.m.)"),
#     y=alt.Y("time",title="time played (s)"),
#     ).properties(title="playtime vs time started")

In [16]:
training, testing = train_test_split(
    main_df,
    test_size=0.25,
    random_state=2000,  # Do not change the random_state
)
X_train = training[["end_hour_of_day"]]  # A single column data frame
y_train = training["session_play_time"]  # A series

X_test = testing[["end_hour_of_day"]]  # A single column data frame
y_test = testing["session_play_time"]  # A series

In [17]:
pipe = make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(),
)

marathon_cv = pd.DataFrame(
    cross_validate(
        estimator=pipe,
        X=X_train,
        y=y_train,
        cv=5,
        scoring="neg_root_mean_squared_error",
        return_train_score=True,
    )
)

marathon_cv

,fit_time,score_time,test_score,train_score
0,0.013936,0.002729,-1.043066,-0.823191
1,0.003008,0.002146,-0.931811,-0.854639
2,0.002923,0.002118,-0.986957,-0.870458
3,0.002924,0.002124,-0.872028,-0.878992
4,0.002941,0.002140,-1.014720,-0.824356


In [18]:
regress("end_hour_of_day","ending hour (hours since 12am)")

{'kneighborsregressor__n_neighbors': 71}


alt.LayerChart(...)

^^^ this graph delete !!!

**Discussion**

(add at end)
our model is eh,

talk about how we cahgned predictors only using one at the beginning

- From our various visualizations we came to some concrete conclusions above player behavior (days and times of highest activity), but nothing that we couldn't guess from how gamers in university spend their spare time.
- The dataset appears highly variable and inconsistent, and there is no clear pattern linking starting time to session duration. Overall, the model's performance is poor but expected given the weak relationship between the predictor and the response.